<font color='black'><div class="alert alert-info"><strong><H2> Student Class Project</H2></strong><br>

In [1]:
import sqlite3                # Required to connect to a local lightweight Python database
import random                 # Required for generating random objects
from datetime import datetime # Required for getting date

**Create a database to hold student records**

In [2]:
conn = sqlite3.connect("Students.db")
cursor = conn.cursor()

cursor.execute("""
CREATE TABLE IF NOT EXISTS students (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    student_id TEXT UNIQUE,
    first_name TEXT,
    last_name TEXT,
    email TEXT UNIQUE,
    gender TEXT,
    courses TEXT
)
""")

conn.commit()
conn.close()

print("Database ready!")

Database ready!


**Generate Student Id**

In [3]:
def generate_student_id():
    year = datetime.now().year
    random_number = random.randint(1000, 9999)
    return f"STU-{year}-{random_number}"

**Check Student id exists**

In [4]:
def student_id_exists(student_id):
    conn = sqlite3.connect("Students.db")
    cursor = conn.cursor()

    cursor.execute("SELECT 1 FROM students WHERE student_id=?", (student_id,)) 
    # id=? is to prevent or avoid sql injection, ? is a placeholder
    exists = cursor.fetchone() is not None

    conn.close()
    return exists

**Generate unique student id**

In [5]:
def generate_unique_student_id():
    student_id = generate_student_id()
    while student_id_exists(student_id): # to avoid creating duplicate student id
        student_id = generate_student_id()
    return student_id

**Generate Student emails**

In [6]:
def generate_email(first_name, last_name, number=None):
    base = f"{first_name.lower()}.{last_name.lower()}"
    if number is None:
        return f"{base}@wl.edu.uk"
    else:
        return f"{base}{number}@wl.edu.uk"

In [7]:
generate_email("DavId", "kamaRa")

'david.kamara@wl.edu.uk'

**Check if email exists**

In [8]:
def email_exists(email):
    conn = sqlite3.connect("Students.db")
    cursor = conn.cursor()

    cursor.execute("SELECT 1 FROM students WHERE email=?", (email,))
    exists = cursor.fetchone() is not None

    conn.close()
    return exists


**Generate unique student email**

In [9]:
def generate_unique_email(first_name, last_name):
    email = generate_email(first_name, last_name)
    counter = 1

    while email_exists(email):
        email = generate_email(first_name, last_name, counter)
        counter += 1
    return email

In [10]:
courses = ["Chemistry", "Mathematics"] # list cannot go into dbase - we need a long string
",".join(courses)

'Chemistry,Mathematics'

In [11]:
courses = ["Chemistry", "Mathematics"] # list cannot go into dbase - we need a long string
"|".join(courses)

'Chemistry|Mathematics'

In [12]:
class Student:
    university = "Woolwich University"
    total_students = 0
    
    def __init__(self, first_name, last_name, gender, courses=None):
        self.first_name = first_name
        self.last_name = last_name
        self.gender = gender
        
        # Auto-generate unique ID & email
        self.student_id = generate_unique_student_id()
        self.email = generate_unique_email(first_name, last_name)

        Student.total_students += 1

        if courses is None:
            self.courses = []
        else:
            self.courses = courses

    def course_string(self):
        """Convert list of courses to comma-separated string as the database can not hold python list"""
        return ",".join(self.courses)

    # ---------------- CRUD OPERATIONS ---------------- #

    def save_to_db(self):
        """CREATE operation"""
        conn = sqlite3.connect("Students.db")
        cursor = conn.cursor()

        cursor.execute("""
            INSERT INTO students 
            (student_id, first_name, last_name, email, gender, courses)
            VALUES (?, ?, ?, ?, ?, ?) 
        """, (self.student_id, self.first_name, self.last_name,
              self.email, self.gender, self.course_string()))

        conn.commit()
        conn.close()
        print(f"Student {self.student_id} saved to database!")

    @staticmethod
    def get_all_students():
        """Show all students records"""
        conn = sqlite3.connect("Students.db")
        cursor = conn.cursor()

        cursor.execute("SELECT student_id, first_name, last_name, email, gender, courses FROM students")
        rows = cursor.fetchall()
        conn.close()

        students = []
        for row in rows:
            student_id, first_name, last_name, email, gender, courses = row
            course_list = courses.split(",") if courses else []
            s = Student(first_name, last_name, gender, course_list)
            s.student_id = student_id       # Override regenerated ID
            s.email = email                 # Override regenerated email
            students.append(s)

        return students

    def update_in_db(self):
        """UPDATE based on the object's student_id"""
        conn = sqlite3.connect("Students.db")
        cursor = conn.cursor()

        cursor.execute("""
            UPDATE students
            SET first_name=?, last_name=?, email=?, gender=?, courses=?
            WHERE student_id=?
        """, (self.first_name, self.last_name, self.email,
              self.gender, self.course_string(), self.student_id))

        conn.commit()
        conn.close()
        print(f"Student {self.student_id} updated!")

    @staticmethod
    def delete_student(student_id):
        """DELETE by student_id"""
        conn = sqlite3.connect("Students.db")
        cursor = conn.cursor()

        cursor.execute("DELETE FROM students WHERE student_id=?", (student_id,))
        conn.commit()
        conn.close()
        print(f"Student {student_id} deleted!")

    # ---------------- Our METHODS from last week, remember! ---------------- #

    def add_course(self, course):
        if course not in self.courses:
            self.courses.append(course)
        else:
            print(f"{self.first_name} is already enrolled in {course}")

    def remove_course(self, course):
        if course in self.courses:
            self.courses.remove(course)
        else:
            print(f"{self.first_name} is not enrolled in {course}")

    def __str__(self):
        return f"""
                    Student ID: {self.student_id}
                    Full Name: {self.first_name} {self.last_name}
                    Email: {self.email}
                    Gender: {self.gender}
                    Courses: {self.courses}
                    University: {Student.university}
                    """

    def __repr__(self):
        return f"Student({self.student_id}, {self.first_name}, {self.last_name})"

    def __len__(self):
        return len(self.courses)


**Create a student**

In [13]:
s1 = Student("Mary", "Adams", "Female", ["Math", "Physics", "Chemistry"])
s1.save_to_db()

Student STU-2025-1940 saved to database!


**Read all records using pandas**

In [14]:
from tabulate import tabulate

In [33]:
import sqlite3
import pandas as pd
from tabulate import tabulate

# Connect to the SQLite database
connection = sqlite3.connect("Students.db")  # Ensure the file name matches your database

# Query the table and load it into a Pandas DataFrame
query = "SELECT * FROM students"  
df = pd.read_sql_query(query, connection)

# Display the DataFrame in a tabulated format
print(tabulate(df, headers='keys', tablefmt='psql'))

# Close the connection
connection.close()

+----+------+---------------+--------------+-------------+-----------------------+----------+----------------------------------+
|    |   id | student_id    | first_name   | last_name   | email                 | gender   | courses                          |
|----+------+---------------+--------------+-------------+-----------------------+----------+----------------------------------|
|  0 |    1 | STU-2025-7462 | Mary         | Adams       | mary.adams@wl.edu.uk  | Female   | Math,Physics,Chemistry           |
|  1 |    2 | STU-2025-1119 | Mary         | Adams       | mary.adams1@wl.edu.uk | Female   | Math,Physics,Chemistry,Economics |
+----+------+---------------+--------------+-------------+-----------------------+----------+----------------------------------+


In [26]:
students = Student.get_all_students()
for s in students:
    print(s)



                    Student ID: STU-2025-6412
                    Full Name: Mary Adams
                    Email: mary.adams@wl.edu.uk
                    Gender: Female
                    Courses: ['Math', ' Physics', ' Chemistry']
                    University: Woolwich University
                    

                    Student ID: STU-2025-4295
                    Full Name: Mary Adams
                    Email: mary.adams1@wl.edu.uk
                    Gender: Female
                    Courses: ['Math', 'Physics', 'Chemistry']
                    University: Woolwich University
                    


In [30]:
s1.remove_course("Economics")
s1.update_in_db()

Mary is not enrolled in Economics
Student STU-2025-1119 updated!


In [31]:
s1.add_course("Economics")
s1.update_in_db()

Student STU-2025-1119 updated!


In [32]:
students = Student.get_all_students()
for s in students:
    print(s)



                    Student ID: STU-2025-7462
                    Full Name: Mary Adams
                    Email: mary.adams@wl.edu.uk
                    Gender: Female
                    Courses: ['Math', 'Physics', 'Chemistry']
                    University: Woolwich University
                    

                    Student ID: STU-2025-1119
                    Full Name: Mary Adams
                    Email: mary.adams1@wl.edu.uk
                    Gender: Female
                    Courses: ['Math', 'Physics', 'Chemistry', 'Economics']
                    University: Woolwich University
                    
